In [1]:
import os
import cv2
import face_recognition

In [2]:
# Path to the folder containing images
folder_path = "Images/"



In [11]:
# Initialize variables to store encodings and labels
known_encodings = []
known_labels = []
label_count = 0

# Set the minimum confidence value for face matching
min_confidence = 0.6  # Adjust this value as needed

# Resize parameters
resize_factor = 0.5  # Adjust this factor to resize images

# Iterate over each image file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(folder_path, filename)
        
        # Load the image
        image = cv2.imread(image_path)
        
        # Resize the image to reduce memory usage
        resized_image = cv2.resize(image, (0, 0), fx=resize_factor, fy=resize_factor)
        
        # Convert the resized image from BGR to RGB (face_recognition uses RGB)
        rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        
        # Find face locations and encodings in the image
        face_locations = face_recognition.face_locations(rgb_image, model="cnn")
        face_encodings = face_recognition.face_encodings(rgb_image, face_locations)
        
        # Assign labels to faces based on their encodings
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Calculate the confidence level of the face detection
            confidence = (right - left) * (bottom - top) / (rgb_image.shape[0] * rgb_image.shape[1])
            
            # Check if the confidence level meets the minimum threshold
            if confidence >= min_confidence:
                # Initialize flag to check if a face has been assigned a label
                face_assigned = False
                
                # Compare the encoding with known encodings
                for i, known_encoding in enumerate(known_encodings):
                    # Compare the face encoding with known encodings
                    distance = face_recognition.face_distance([known_encoding], face_encoding)
                    
                    # Check if the distance is within the confidence threshold
                    if distance < min_confidence:
                        # If a match is found, assign the corresponding label
                        face_assigned = True
                        label = known_labels[i]
                        break
                
                # If face is not assigned a label, assign a new unique label
                if not face_assigned:
                    label_count += 1
                    label = label_count
                    
                    # Store the encoding and label
                    known_encodings.append(face_encoding)
                    known_labels.append(label)
                    
                # Draw rectangle around the face and annotate with label
                # Scale back the face locations to match original image size
                scaled_top = int(top / resize_factor)
                scaled_right = int(right / resize_factor)
                scaled_bottom = int(bottom / resize_factor)
                scaled_left = int(left / resize_factor)
                
                cv2.rectangle(image, (scaled_left, scaled_top), (scaled_right, scaled_bottom), (0, 255, 0), 2)
                cv2.putText(image, f"Face {label}", (scaled_left, scaled_top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save the annotated image
        annotated_image_path = os.path.join("annotated_images", filename)
        cv2.imwrite(annotated_image_path, image)

# Save the labels and corresponding image filenames
with open("labels.txt", "w") as file:
    for filename, label in zip(os.listdir(folder_path), known_labels):
        file.write(f"{filename}: {label}\n")

MemoryError: bad allocation